In [87]:
import pandas as pd
from rapidfuzz import fuzz,process
charge_capture_df=pd.read_excel("Charge_Capture_-_Jul_20th_-_Aug_2nd.xlsx")
sorted_fac = sorted(charge_capture_df['Facility'].unique())

import re
metadata = {}
pattern = re.compile(r'\b9930\d{2}\b')  # Example pattern for 6-digit codes starting with 9930
cpt_time_dict = {
    "99304": 60,
    "99305": 75,
    "99306": 90,
    "99307": 15,
    "99308": 25,
    "99309": 35,
    "99310": 45
}
Facilites_groups=charge_capture_df.groupby('Facility', as_index=False)
sites_tracker=pd.read_excel("D:\payroll\Site_Tracker_1754578592.xlsx",header=2)
sites_tracker=sites_tracker[["Name","Chain"]]
print(len(sorted_fac))

<>:19: SyntaxWarning: invalid escape sequence '\p'
<>:19: SyntaxWarning: invalid escape sequence '\p'
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_31572\952682461.py:19: SyntaxWarning: invalid escape sequence '\p'
  sites_tracker=pd.read_excel("D:\payroll\Site_Tracker_1754578592.xlsx",header=2)


200


In [ ]:
from collections import defaultdict
stopwords = {"of", "the", "at","rehabilitation and nursing center","nursing and rehabilitation center"}

def normalize_name(name):
    # Convert to lowercase and remove stopwords
    if not isinstance(name, str):
        name = str(name)
    name = name.replace("-", "")  # Remove hyphens
    name = name.replace("&", "and")  # Replace ampersands with 'and'
    name=name.replace("(M)","")
    words = name.lower()
    return words

# Ensure all values in 'Name' column are strings
sites_tracker['Name'] = sites_tracker['Name'].astype(str)

# Example: group by the first word in the facility name
grouped_facilities = defaultdict(list)
for fac in sorted_fac:
    normalized_fac = normalize_name(fac)
    normalized_site = sites_tracker['Name'].apply(normalize_name)
    match = process.extract(normalized_fac, normalized_site, scorer=fuzz.partial_ratio, limit=5)
    if match[0][1] > 94:
        filtered = sites_tracker[sites_tracker['Name'].str.contains(match[0][0], case=False, na=False)]
        if not filtered.empty:
            key = filtered['Chain'].values[0]
            grouped_facilities[key].append((match[0][0], match[0][1], fac))
    else:
        key = fac.split()[0]
        # print(fac)
        if key.lower() in stopwords:
            key = fac.split()[1]
        grouped_facilities[key].append(("none", "  ", fac))
    
corporate_groups = {k: v for k, v in grouped_facilities.items() if len(v) > 1}
single_facilities = {k: v for k, v in grouped_facilities.items() if len(v) == 1}

# Save corporate_groups and single_facilities to an Excel file
with pd.ExcelWriter("grouped_facilities.xlsx") as writer:
    corporate_groups_df = pd.DataFrame([(key, facility[0], facility[1], facility[2]) for key, facilities in corporate_groups.items() for facility in facilities], columns=["Group", "Match Name", "Score", "Facility Name"])
    single_facilities_df = pd.DataFrame([(key, facility[0], facility[1], facility[2]) for key, facilities in single_facilities.items() for facility in facilities], columns=["Group", "Match Name", "Score", "Facility Name"])

    corporate_groups_df.to_excel(writer, sheet_name="Corporate Groups", index=False)
    single_facilities_df.to_excel(writer, sheet_name="Single Facilities", index=False)

print("Corporate groups and single facilities have been saved to grouped_facilities.xlsx")

Aliya of Highwood
Aliya of Homewood
Aliya of Oak Lawn
Aliya of Palos Park
Alta Rehab at Fairmont (M)
Alta Rehab at Oak Brook
Arcadia Care Peoria Heights
Beecher Manor Nursing and Rehab Center
Crestwood Rehabilitation Center, LLC
Crown Point Health Campus
Elevate Care Abington
Elevate Care Palos Heights
Elevate Care Windsor Park
Elevate Care of South Holland
Encore Village SNF (M)
Fountain Blue Rehab and Nursing
Harbor Health & Rehab (M)
Haven of St. Elmo LLC
Highland Operations LLC
Highland Village Skilled Care, LLC
Kingston Care Center of Sylvania
Kingston Rehabilitation of Perrysburg
Lakes Care Center -SNF
Lakewood Nursing and Rehabilitation Center
Lemont Nursing and Rehab Center
Lincolnshire Health Care Center
Munster Med Inn
Neighbors Care Center (M)
OSF Home Health
Seacrest Rehabilitation and Nursing Center
Spring Mill Health Campus
St. Francis - SNF
St. James Manor and Villas
Test Facility
Test Facility Updated 
Corporate groups and single facilities have been saved to grouped_fa

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_31572\36378903.py:24: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered = sites_tracker[sites_tracker['Name'].str.contains(match[0][0], case=False, na=False)]


In [42]:
for key, facilities in corporate_groups.items():
    metadata[key] = {}  # Initialize the dictionary for this key
    metadata[key]['Facility_Data'] = {}
    corp_data = pd.DataFrame()
    facility_summary = []
    for facility in facilities:
        print(f"Processing Corporate Group: {key}, Facility: {facility}")
        group = Facilites_groups.get_group(facility).copy()
        group['CPT Codes'] = group['CPT Codes'].astype(str).str.split(',')
        group = group.explode('CPT Codes')
        group['CPT Codes'] = group['CPT Codes'].str.strip()
        Facility_Level = group.groupby(['Provider', 'CPT Codes', 'DOS']).size().reset_index(name='Count')
        Facility_Level['Minutes'] = Facility_Level['CPT Codes'].astype(str).map(cpt_time_dict) * Facility_Level['Count']
        # Remove rows where Minutes == 0 or is NaN
        Facility_Level = Facility_Level[(Facility_Level['Minutes'] != 0) & (Facility_Level['Minutes'].notna())]
        Facility_Level_result = Facility_Level.groupby(['Provider', 'DOS'], as_index=False)['Minutes'].sum()
        Facility_Level_result['Hours'] = Facility_Level_result['Minutes'] / 60
        # Group by Provider for this facility and sum hours
        provider_hours = Facility_Level_result.groupby('Provider', as_index=False)['Hours'].sum()
        provider_hours['Facility'] = facility
        facility_summary.append(provider_hours)
        metadata[key]['Facility_Data'][facility] = Facility_Level_result
    # Concatenate all facility summaries
    summary_df = pd.concat(facility_summary, ignore_index=True)
    # Group by Facility, aggregate clinicians and sum hours
    summary_grouped = summary_df.groupby('Facility').agg({
        'Provider': lambda x: '\n'.join(x),
        'Hours': 'sum'
    }).reset_index()
    summary_grouped = summary_grouped.rename(columns={'Provider': 'Clinician'})
    metadata[key]['Corp_data'] = summary_grouped
    print(summary_grouped)
    break

Processing Corporate Group: Omnia / Citadel, Facility: AMBD Property LLC dba Amberwood Care Centre (M)
Processing Corporate Group: Omnia / Citadel, Facility: Citadel Care Center - Sterling LLC
Processing Corporate Group: Omnia / Citadel, Facility: PA Peterson at the Citadel
Processing Corporate Group: Omnia / Citadel, Facility: Pavilion of Ottawa (M)
                                          Facility         Clinician  \
0  AMBD Property LLC dba Amberwood Care Centre (M)  Melissa Townsend   
1               Citadel Care Center - Sterling LLC  Katelin Breuning   
2                       PA Peterson at the Citadel  Melissa Townsend   
3                           Pavilion of Ottawa (M)     Tamara  Baird   

       Hours  
0  15.833333  
1  10.833333  
2  32.083333  
3  47.166667  


Corporate groups and single facilities have been saved to grouped_facilities.xlsx


In [ ]:
group = Facilites_groups.get_group( 'Accolade Healthcare of Pontiac LLC')
provider=group[['Provider', 'CPT Codes', 'DOS']]
provider['CPT Codes'] = provider['CPT Codes'].astype(str).str.split(',')
provider_exploded = provider.explode('CPT Codes')
provider_exploded['CPT Codes'] = provider_exploded['CPT Codes'].str.strip()
provider_exploded
counts = group[['Provider', 'CPT Codes']].value_counts().reset_index(name='Count')
counts

In [ ]:
filtered = provider_exploded[(provider_exploded['Provider'] == 'Hina Patel')]
cpt_time_dict = {
    "99304": 60,
    "99305": 75,
    "99306": 90,
    "99307": 15,
    "99308": 25,
    "99309": 35,
    "99310": 45
}
sum=150+(13*35)

In [ ]:
filtered

In [ ]:
metadata["Accolade"]

In [ ]:

new_PR=pd.read_excel("D:\payroll\Payroll History (18) (1).xlsx")

In [ ]:
import pandas as pd
sites_tracker=pd.read_excel("D:\payroll\Site_Tracker_1754578592.xlsx",header=2)
sites_tracker=sites_tracker[["Name","Chain"]]

In [ ]:
sites_tracker.head()

In [ ]:
from rapidfuzz import fuzz,process

for fac in sorted_fac:
    match = process.extract(fac, sites_tracker['Name'], scorer=fuzz.partial_ratio, limit=5)
    if not match:
        print(f"Facility: {fac}")
        print("No matches found")
    else:
        # print(f"Facility: {fac}")
        if match[0][1] < 80:
            print(f"Facility: {fac}")
            print("top matche", match[0])
            print("sites_tracker matches", sites_tracker[sites_tracker['Name'].str.contains(match[0][0], case=False, na=False)]['Chain'].values[0])
            print("-" * 40)

In [6]:

from collections import defaultdict
from rapidfuzz import fuzz,process
import math
import pandas as pd
def build_metadata(charge_capture_df, corporate_groups, single_facilities):
    cpt_time_dict = {
        "99304": 60,
        "99305": 75,
        "99306": 90,
        "99307": 15,
        "99308": 25,
        "99309": 35,
        "99310": 45
    }
    Facilites_groups = charge_capture_df.groupby('Facility', as_index=False)
    metadata = {}
    for key, facilities in corporate_groups.items():
        corp_data = pd.DataFrame()
        facility_summary = []
        facility_data_dict = {}
        for facility in facilities:
            group = Facilites_groups.get_group(facility).copy()
            group.loc[:, 'CPT Codes'] = group['CPT Codes'].astype(str).str.split(',')
            group = group.explode('CPT Codes')
            group['CPT Codes'] = group['CPT Codes'].str.strip()
            Facility_Level = group.groupby(['Provider', 'CPT Codes', 'DOS']).size().reset_index(name='Count')
            Facility_Level['Minutes'] = Facility_Level['CPT Codes'].astype(str).map(cpt_time_dict) * Facility_Level['Count']
            # Remove rows where Minutes == 0 or is NaN
            Facility_Level = Facility_Level[(Facility_Level['Minutes'] != 0) & (Facility_Level['Minutes'].notna())]
            if Facility_Level.empty:
                continue  # Skip this facility if no valid entries remain
            Facility_Level_result = Facility_Level.groupby(['Provider', 'DOS'], as_index=False)['Minutes'].sum()
            # Round up each clinician's hours to the nearest 0.5
            def round_up_half(x):
                return math.ceil(x * 2) / 2
            Facility_Level_result['Hours'] = Facility_Level_result['Minutes'].apply(lambda x: round_up_half(x / 60))
            # Group by Provider for this facility and sum hours (sum after rounding up each row)
            provider_hours = Facility_Level_result.groupby('Provider', as_index=False)['Hours'].sum()
            provider_hours['Facility'] = facility
            facility_summary.append(provider_hours)
            facility_data_dict[facility] = Facility_Level_result
        # Only add to metadata if there is at least one valid facility
        if not facility_summary:
            continue
        metadata[key] = {}
        metadata[key]['Facility_Data'] = facility_data_dict
        summary_df = pd.concat(facility_summary, ignore_index=True)
        # Group by Facility, aggregate clinicians and sum hours
        summary_grouped = summary_df.groupby('Facility').agg({
            'Provider': lambda x: '\n'.join(x),
            'Hours': 'sum'
        }).reset_index()
        summary_grouped = summary_grouped.rename(columns={'Provider': 'Clinician'})
        metadata[key]['Corp_data'] = summary_grouped
    for key, facilities in single_facilities.items():
        facility_name = facilities[0]  # Single facility case
        facility_name = facility_name.strip().split("and")[0]  # Ensure no leading/trailing spaces
        corp_data = pd.DataFrame()
        facility_summary = []
        facility_data_dict = {}
        for facility in facilities:
            group = Facilites_groups.get_group(facility).copy()
            group.loc[:, 'CPT Codes'] = group['CPT Codes'].astype(str).str.split(',')
            group = group.explode('CPT Codes')
            group['CPT Codes'] = group['CPT Codes'].str.strip()
            Facility_Level = group.groupby(['Provider', 'CPT Codes', 'DOS']).size().reset_index(name='Count')
            Facility_Level['Minutes'] = Facility_Level['CPT Codes'].astype(str).map(cpt_time_dict) * Facility_Level['Count']
            # Remove rows where Minutes == 0 or is NaN
            Facility_Level = Facility_Level[(Facility_Level['Minutes'] != 0) & (Facility_Level['Minutes'].notna())]
            if Facility_Level.empty:
                continue  # Skip this facility if no valid entries remain
            Facility_Level_result = Facility_Level.groupby(['Provider', 'DOS'], as_index=False)['Minutes'].sum()
            # Round up each clinician's hours to the nearest 0.5
            def round_up_half(x):
                return math.ceil(x * 2) / 2
            Facility_Level_result['Hours'] = Facility_Level_result['Minutes'].apply(lambda x: round_up_half(x / 60))
            # Group by Provider for this facility and sum hours (sum after rounding up each row)
            provider_hours = Facility_Level_result.groupby('Provider', as_index=False)['Hours'].sum()
            provider_hours['Facility'] = facility
            facility_summary.append(provider_hours)
            facility_data_dict[facility] = Facility_Level_result
        # Only add to metadata if there is at least one valid facility
        if not facility_summary:
            continue
        metadata[facility_name] = {}
        metadata[facility_name]['Facility_Data'] = facility_data_dict
        summary_df = pd.concat(facility_summary, ignore_index=True)
        # Group by Facility, aggregate clinicians and sum hours
        summary_grouped = summary_df.groupby('Facility').agg({
            'Provider': lambda x: '\n'.join(x),
            'Hours': 'sum'
        }).reset_index()
        summary_grouped = summary_grouped.rename(columns={'Provider': 'Clinician'})
        metadata[facility_name]['Corp_data'] = summary_grouped
    return metadata

def group_facilities(charge_capture_df,sites_tracker):
    sorted_fac = sorted(charge_capture_df['Facility'].unique())
    stopwords = {"of", "the", "at","rehabilitation and nursing center","nursing and rehabilitation center"}

    def normalize_name(name):
        # Convert to lowercase and remove stopwords
        if not isinstance(name, str):
            name = str(name)
        name = name.replace("-", "")  # Remove hyphens
        name = name.replace("&", "and")  # Replace ampersands with 'and'
        name=name.replace("(M)","")
        words = name.lower()
        return words
    sites_tracker['Name'] = sites_tracker['Name'].astype(str)
    grouped_facilities = defaultdict(list)
    for fac in sorted_fac:
        normalized_fac = normalize_name(fac)
        normalized_site = sites_tracker['Name'].apply(normalize_name)
        match = process.extract(normalized_fac, normalized_site, scorer=fuzz.partial_ratio, limit=5)
        if match[0][1] > 94:
            filtered = sites_tracker[sites_tracker['Name'].str.contains(match[0][0], case=False, na=False)]
            if not filtered.empty:
                key = filtered['Chain'].values[0]
                grouped_facilities[key].append(fac)
        else:
            key = fac.split()[0]
            # print(fac)
            if key.lower() in stopwords:
                key = fac.split()[1]
            grouped_facilities[key].append(fac)
    corporate_groups = {k: v for k, v in grouped_facilities.items() if len(v) > 1}
    single_facilities = {k: v for k, v in grouped_facilities.items() if len(v) == 1}

    return corporate_groups, single_facilities

charge_capture_df=pd.read_excel(r"D:\payroll\uploads\Charge_Capture_-_Jul_20th_-_Aug_2nd.xlsx")
sites_tracker=pd.read_excel(r"D:\payroll\static\Site_Tracker_1754578592.xlsx",header=2)
sites_tracker=sites_tracker[["Name","Chain"]]
corporate_groups, single_facilities = group_facilities(charge_capture_df,sites_tracker)
metadata = build_metadata(charge_capture_df, corporate_groups, single_facilities)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_6064\1018505352.py:117: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered = sites_tracker[sites_tracker['Name'].str.contains(match[0][0], case=False, na=False)]


In [11]:
metadata

{'Omnia / Citadel': {'Facility_Data': {'AMBD Property LLC dba Amberwood Care Centre (M)':            Provider         DOS  Minutes  Hours
   0  Melissa Townsend  07/22/2025    275.0    5.0
   1  Melissa Townsend  07/24/2025    250.0    4.5
   2  Melissa Townsend  07/29/2025    275.0    5.0
   3  Melissa Townsend  07/31/2025    150.0    2.5,
   'Citadel Care Center - Sterling LLC':            Provider         DOS  Minutes  Hours
   0  Katelin Breuning  07/21/2025      300    5.0
   1  Katelin Breuning  07/24/2025      350    6.0,
   'PA Peterson at the Citadel':            Provider         DOS  Minutes  Hours
   0  Melissa Townsend  07/22/2025    425.0    7.5
   1  Melissa Townsend  07/24/2025    500.0    8.5
   2  Melissa Townsend  07/29/2025    375.0    6.5
   3  Melissa Townsend  07/31/2025    625.0   10.5,
   'Pavilion of Ottawa (M)':         Provider         DOS  Minutes  Hours
   0  Tamara  Baird  07/21/2025    600.0   10.0
   1  Tamara  Baird  07/24/2025    610.0   10.5
   2  Tam